In [48]:
import dirichletcal
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.calibration import CalibratedClassifierCV
from sklearn.isotonic import IsotonicRegression
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import average_precision_score
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
import pandas as pd
import math
import optuna
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from NFLUtils import NFLUtils
nfl_utils = NFLUtils()
from MLBUtils import MLBUtils
mlb_utils = MLBUtils()

%matplotlib inline

import logging
# Set up logging
logging.basicConfig(level=logging.INFO)
# Set pandas display option to show all columns
pd.set_option('display.max_columns', None)

# XGBoost 
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold, cross_val_score, cross_val_predict
import seaborn as sns # confusion matrix

from dirichletcal.calib.fulldirichlet import FullDirichletCalibrator

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import jax
jax.config.update("jax_platform_name", "cpu")

In [49]:
def set_all_seeds(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Load CSV


In [50]:
# df = pd.read_csv("./ttennisData/gen/TTCupSliding10_100_30.csv", index_col=False, low_memory=False) # H_won
df = pd.read_csv("./ttennisData/gen/TTCupSliding10_20_30.csv", index_col=False, low_memory=False) # G5

print(df.shape)
# LEFT SHIFT DF to simulate a different dataset basically
left_shift = 0
df = df[:df.shape[0]-left_shift]

df['H_won'] = df['Total_P1'] > df['Total_P2']
df['Under_74'] = abs(df['Over_74'] - 1)

# Remove the performance set
test_performance_size = 300
test_performance_size = 10000
test_performance_df = df[df.shape[0]-test_performance_size:]
df = df[:df.shape[0]-test_performance_size]
print(df.shape)
df.info()

(268091, 52)
(258091, 54)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258091 entries, 0 to 258090
Data columns (total 54 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        258091 non-null  int64  
 1   Date              258091 non-null  int64  
 2   Player1           258091 non-null  int64  
 3   Player2           258091 non-null  int64  
 4   Sets_P1           258091 non-null  float64
 5   Sets_P2           258091 non-null  float64
 6   G1_P1             258091 non-null  float64
 7   G1_P2             258091 non-null  float64
 8   G2_P1             258091 non-null  float64
 9   G2_P2             258091 non-null  float64
 10  G3_P1             258091 non-null  float64
 11  G3_P2             258091 non-null  float64
 12  G4_P1             258091 non-null  float64
 13  G4_P2             258091 non-null  float64
 14  G5_P1             258091 non-null  float64
 15  G5_P2             258091 non-null  float64

In [51]:
checkEmpty = df[df["Total_Score"] == 0]
print(checkEmpty.shape)

(0, 54)


In [52]:

# Load data
x_col = [
    'Sets_P1',
    'Sets_P2',
    # 'Total_P1', # Careful re: leakage
    # 'Total_P2', # Careful re: leakage
    'Total_Allowed_P1',
    'Total_Allowed_P2',
    'pythagorean_P1',
    'pythagorean_P2',
    'D_Sets',
    'D_Odds',
#    'No_Odds', # Removed in EDA
    # 'D_Total', # Careful re: leakage
    'D_Total_Allowed'
]



# For H_won
# x_col = ['D_Odds', 'pythagorean_P1', 'pythagorean_P2', 'elo_P1', 'elo_P2', 'Total_Allowed_P1', 'G3_P2', 'G1_P1', 'Sets_P2', 'G1_P2', 'Total_Allowed_P2', 'G3_P1', 'G2_P1', 'G2_P2', 'Sets_P1']
x_col = ['D_Odds', 'pythagorean_P1', 'pythagorean_P2', 'elo_P2', 'elo_P1', 'D_G3', 'D_G1', 'Sets_P2', 'Win_P1', 'Total_Allowed_P1', 'D_G2', 'Win_P2', 'Sets_Allowed_P2', 'Sets_Allowed_P1', 'Total_Allowed_P2', 'Sets_P1']
x_col = ['D_Odds', 'pythagorean_P1', 'pythagorean_P2', 'elo_P2', 'elo_P1', 'D_G3', 'D_G1', 'Sets_P2', 'Win_P1', 'Total_Allowed_P1', 'D_G2', 'Win_P2', 'Total_Allowed_P2', 'Sets_P1']
x_col = ['G1_P2', 'D_G1', 'D_G2', 'D_G3', 'Win_P1', 'pythagorean_P1', 'pythagorean_P2', 'elo_P1', 'elo_P2', 'Total_Allowed_P1', 'Total_Allowed_P2', 'Sets_Allowed_P2', 'D_Odds', 'Sets_P2', 'G2_P2', 'Win_P2', 'Sets_Allowed_P1']

# For G45
#_col = ['G4_P2', 'G4_P1', 'D_Odds', 'G5_P1', 'elo_P2', 'Total_Allowed_P2', 'Total_Allowed_P1', 'elo_P1', 'pythagorean_P1', 'Sets_P2', 'G5_P2', 'G1_P1', 'G3_P1', 'G3_P2', 'Sets_P1', 'G2_P2', 'G2_P1', 'pythagorean_P2', 'G1_P2']

# For G5
#x_col = ['G5_P1', 'G5_P2', 'Total_Allowed_P2', 'Total_Allowed_P1', 'elo_P2', 'G4_P1', 'G4_P2', 'elo_P1', 'G1_P1', 'pythagorean_P1', 'D_Odds', 'Sets_P1', 'G2_P1', 'G3_P1', 'pythagorean_P2', 'Sets_P2']
#x_col = ['G5_P1', 'Total_Allowed_P2', 'G5_P2', 'elo_P2', 'Total_Allowed_P1', 'G4_P1', 'Total_Avg_P2', 'D_Odds', 'Total_Avg_P1', 'elo_P1', 'D_G2', 'G1_P1', 'G3_P1', 'pythagorean_P1', 'pythagorean_P2']
# x_col = ['G4_P1', 'G4_P2', 'G5_P1', 'G5_P2', 'elo_P1', 'elo_P2', 'Total_Avg_P1', 'Total_Avg_P2', 'Total_Allowed_P1', 'Total_Allowed_P2', 'D_Odds', 'G1_P1', 'pythagorean_P2']
x_col = ['G4_P1', 'G4_P2', 'G5_P1', 'elo_P1', 'elo_P2', 'Total_Avg_P1', 'Total_Allowed_P1', 'Total_Allowed_P2', 'D_Odds', 'G1_P1', 'G5_P2', 'Total_Avg_P2']

# For over 74.5
#x_col = ['Total_Allowed_P2', 'Total_Allowed_P1', 'G4_P2', 'G4_P1', 'D_Odds', 'G3_P1', 'Sets_P1', 'G3_P2', 'G5_P1', 'pythagorean_P1', 'G2_P1', 'pythagorean_P2']
#x_col = ['Total_Allowed_P2', 'Total_Allowed_P1', 'Total_Avg_P1', 'G4_P2', 'Total_Avg_P2', 'G4_P1', 'Player1', 'G5_P1', 'Player2', 'G3_P1', 'G2_P1', 'G2_P2', 'G3_P2', 'Sets_Allowed_P1', 'Date', 'Sets_P1', 'G5_P2', 'Sets_P2', 'G1_P2', 'Win_P1', 'Sets_Allowed_P2', 'G1_P1', 'Win_P2']
#x_col = ['Total_Allowed_P2', 'Total_Allowed_P1', 'Total_Avg_P1', 'G4_P2', 'D_Odds', 'Total_Avg_P2', 'G4_P1', 'Player1', 'Player2', 'elo_P1', 'pythagorean_P2', 'pythagorean_P1']

# y_prediction = 'Over_74'
# y_prediction = 'Under_74'
y_prediction = 'G5'
# y_prediction = 'G45'
# y_prediction = 'G4'
# y_prediction = 'H_won'

y_col = [y_prediction]
x = df[x_col]
y = df[y_col]

perf_conts_df = test_performance_df[x_col]
perf_y_df = test_performance_df[y_col]


# Create an array of continuous values

Numpy array 'conts' containing stack of each continuous column

In [53]:
conts = np.stack([x[col].values for col in list(x.columns)], 1)
y_col = np.stack([y[col].values for col in list(y.columns)], 1)

perf_conts = np.stack([perf_conts_df[col].values for col in list(perf_conts_df.columns)], 1)
perf_y_col = np.stack([perf_y_df[col].values for col in list(perf_y_df.columns)], 1)

conts_train = conts
y_train = y_col

In [54]:
def convert_probas(probas, local_y_col):
    """
    Using y_col, this converts 2d array of probas into a format that works with the backtest calculation.
    That is, 
    
    probas: 2d array with first element being predicted visitor odds & other being predicted home odds
    y_col: ['H_Won', 'H_start_odds', 'V_start_odds', 'Home_Team_Id', 'Visitor_Team_Id', 'combined']
    """
    # Convert to decimal odds, apply 5% vig estimate
    local_y_col[:,1] = (1 / local_y_col[:,1]) / 1.05
    local_y_col[:,2] = (1 / local_y_col[:,2]) / 1.05
    # probas[:,1] = (1 / probas[:,1])
    # probas[:,0] = (1 / probas[:,0])
    override_req = True
    
    # Set to 0 if < the model's implied probability
    bet_res = []
    for i in range(len(probas)):
        if (probas[i,1] > local_y_col[i,1] or override_req == True) and probas[i,1] > probas[i,0]:
            bet_res.append(probas[i,1]) # Convert back to decimal odds
        elif (probas[i,0] > local_y_col[i,2] or override_req == True) and probas[i,0] > probas[i,1]:
            bet_res.append((-1*probas[i,0])) # Convert back to decimal odds
        else:
            print(f"{probas[i,:]} local ycol = {local_y_col[i,:]}")
            bet_res.append(0)
    return np.array(bet_res)

In [55]:
# conts_train.info()

### Old approach (not based on calibration)

In [56]:
# from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
print(f"BEFORE conts_train: {conts_train.shape}, y_train: {y_col.shape}")
# conts_train, y_col = SMOTE().fit_resample(conts_train, y_col)
# conts_train, y_col = rus.fit_resample(conts_train, y_col)
print(f"AFTER conts_train: {conts_train.shape}, y_train: {y_col.shape}")

BEFORE conts_train: (258091, 12), y_train: (258091, 1)
AFTER conts_train: (258091, 12), y_train: (258091, 1)


In [57]:
# y_col = y_col.reshape(-1, 1)
print(f"AFTER reshape: {conts_train.shape}, y_train: {y_col.shape}")

AFTER reshape: (258091, 12), y_train: (258091, 1)


In [58]:
# print(y_col)

In [60]:
def objective(trial, override_params=None):
    """
    When doing optuna training: trial is defined, override_params left as None
    when recreating model using best parameters (override_params): trial is None, override_params are defined
    """
    undersample_preds = ['G5f', 'G4', 'G45f', 'G54']
    
    is_training = trial != None
    # --- Suggest hyperparameters ---
    if is_training:
        other_eval_metric = ['error', 'auc', 'aucpr', 'logloss']
        other_eval_metric = ['error']
        if y_prediction in undersample_preds:
            other_eval_metric = ['auc', 'aucpr']
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 20, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),  # Add L1 reg
            'gamma': trial.suggest_float('gamma', 0, 5.0),  # Add min split loss
            'other_eval_metric': trial.suggest_categorical('other_eval_metric', other_eval_metric),

            # ONLY FOR G4, G5!!
            'scale_pos_weight':trial.suggest_float('scale_pos_weight', 2.3, 2.35, log=True)
        }
        
    else:
        params=override_params
    
    # For imbalanced datasets (IGNORED BECAUSE OF SMOTE)
    #if y_prediction == 'G5' or y_prediction == 'G4':
    #    scale_weight = sum(y_col == 0) / sum(y_col == 1)
    #    params['scale_pos_weight'] = scale_weight[0] # Under X games / At X games
    
    # Training set to fit XGB
    # Test set to fit Calibrator
    # Holdout set (from test set) to calculate loss
    if is_training:
        loss = []
        kf = KFold(n_splits=9, shuffle=False)
        for train_index, test_index in kf.split(conts_train):
            X_clas = conts_train[train_index]
            X_cal = conts_train[test_index]
    
            y_clas = y_col[train_index].ravel() # ravel=flatten to (n,)
            y_cal = y_col[test_index].ravel()
            
            X_cal, X_holdout, y_cal, y_holdout = train_test_split(X_cal, y_cal, test_size=0.1, shuffle=False)
            # Split a val set from X_clas for early stopping (time-aware)
            X_clas, X_clas_eval, y_clas, y_clas_eval = train_test_split(X_clas, y_clas, test_size=0.1, shuffle=False)
    
            # print(f"X_clas {X_clas.shape} X_cal {X_cal.shape} X_clas_eval {X_clas_eval.shape} X_holdout {X_holdout.shape}")
            
            # Undersample training data
            if y_prediction in undersample_preds:
                # print(f"BEFORE reshape: {X_clas.shape}, y_train: {y_clas.shape}")
                X_clas, y_clas = rus.fit_resample(X_clas, y_clas)
                y_clas = y_clas.reshape(-1,1)
                # print(f"AFTER reshape: {X_clas.shape}, y_train: {y_clas.shape}")
            # Create classifier & fit on test set
            classifier = XGBClassifier(
                objective='binary:logistic',
                # eval_metric='error', # UNCOMMENT FOR Over/Under 74.5
                eval_metric=params['other_eval_metric'],
                early_stopping_rounds=20,
                **{k: v for k, v in params.items() if k.startswith('other_') == False}
            )
            # sample_weight = compute_sample_weight('balanced', y_clas)
            #print(f"sample_weight ${sample_weight}")
            classifier.fit(
                X_clas, y_clas,
                #sample_weight=sample_weight,
                eval_set=[(X_clas_eval, y_clas_eval)],
                verbose=False
            )
            
            # Create calibrator (Note: Ideal to have 5k+ records)
            calibrator = IsotonicRegression(out_of_bounds='clip')
            X_cal = classifier.predict_proba(X_cal)[:,1] # has to be 1D to match y_test
            calibrator.fit(X_cal, y_cal)
    
            # Use holdout set to calculate loss
            X_holdout = classifier.predict_proba(X_holdout)[:,1]
            X_holdout = calibrator.predict(X_holdout)

            # Use AUC-PR if G5 or G4
            holdout_loss = log_loss(y_holdout, X_holdout)
            # holdout_loss = calc_profit(y_holdout, X_holdout)
            
            # if y_prediction == 'G5' or y_prediction == 'G4':
            #    holdout_loss = average_precision_score(y_holdout, X_holdout)
            loss.append(holdout_loss)

        return np.mean(loss)

    
    # ----- When not is_training, train and return classifier, calibrator -----
    # Time-based split: first 90% for clas, last 10% for cal (assumes sorted by time)
    X_clas, X_cal, y_clas, y_cal = train_test_split(conts_train, y_col, test_size=0.1, shuffle=False)
    # Split a val set from X_clas for early stopping (time-aware)
    X_clas, X_clas_eval, y_clas, y_clas_eval = train_test_split(X_clas, y_clas, test_size=0.1, shuffle=False)
    y_clas = y_clas.ravel()
    y_clas_eval = y_clas_eval.ravel()
    y_cal = y_cal.ravel()
    
    # Create classifier
    classifier = XGBClassifier(
        objective='binary:logistic',
        eval_metric=params['other_eval_metric'],
        early_stopping_rounds=20,
        **{k: v for k, v in params.items() if k.startswith('other_') == False}
    )

    # Undersample training data
    if y_prediction in undersample_preds:
        # print(f"BEFORE reshape: {X_clas.shape}, y_train: {y_clas.shape}")
        X_clas, y_clas = rus.fit_resample(X_clas, y_clas)
        y_clas = y_clas.reshape(-1,1)
        # print(f"AFTER reshape: {X_clas.shape}, y_train: {y_clas.shape}")
        
    sample_weight = compute_sample_weight('balanced', y_clas)
    classifier.fit(X_clas, y_clas, sample_weight=sample_weight, eval_set=[(X_clas_eval, y_clas_eval)], verbose=False)

    # Create calibrator, train on classifier predictions
    calibrator = IsotonicRegression(out_of_bounds='clip')
    X_cal = classifier.predict_proba(X_cal)[:,1]
    calibrator.fit(X_cal, y_cal)
    return classifier, calibrator
    

def print_callback(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value}")
    print(f"Best trial so far: {study.best_trial.number}, value: {study.best_trial.value}")
    
    

In [ ]:
study = optuna.create_study(
    direction='minimize',
    pruner=optuna.pruners.HyperbandPruner(min_resource=1, max_resource=1000)
)

study.optimize(objective, n_trials=2000, callbacks=[print_callback])
print("Best trial:")
trial = study.best_trial
print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-25 17:32:14,598] A new study created in memory with name: no-name-30ba1e2d-5a70-4ea6-a395-bfb11163c53d
[I 2025-11-25 17:32:16,367] Trial 0 finished with value: 0.6050094966589253 and parameters: {'learning_rate': 0.055846963383206334, 'max_depth': 6, 'n_estimators': 641, 'subsample': 0.9186105581359086, 'colsample_bytree': 0.6275923419999557, 'min_child_weight': 16, 'reg_lambda': 2.6892460225396486, 'reg_alpha': 0.0004705207388582577, 'gamma': 3.362198483029681, 'other_eval_metric': 'error', 'scale_pos_weight': 2.338604390079665}. Best is trial 0 with value: 0.6050094966589253.


Trial 0 finished with value: 0.6050094966589253
Best trial so far: 0, value: 0.6050094966589253


[I 2025-11-25 17:32:17,933] Trial 1 finished with value: 0.604017553269665 and parameters: {'learning_rate': 0.2137869576302142, 'max_depth': 3, 'n_estimators': 348, 'subsample': 0.9997123670034366, 'colsample_bytree': 0.9831200297064397, 'min_child_weight': 12, 'reg_lambda': 0.25873908374318816, 'reg_alpha': 0.006364530741137008, 'gamma': 1.4553437762686388, 'other_eval_metric': 'error', 'scale_pos_weight': 2.3036898330740367}. Best is trial 1 with value: 0.604017553269665.


Trial 1 finished with value: 0.604017553269665
Best trial so far: 1, value: 0.604017553269665


[I 2025-11-25 17:32:20,382] Trial 2 finished with value: 0.6050220301400763 and parameters: {'learning_rate': 0.03352251086594253, 'max_depth': 7, 'n_estimators': 478, 'subsample': 0.5445168161198476, 'colsample_bytree': 0.7746360445455651, 'min_child_weight': 6, 'reg_lambda': 0.020578861993726983, 'reg_alpha': 0.03219344259104641, 'gamma': 2.828546074726108, 'other_eval_metric': 'error', 'scale_pos_weight': 2.3396782258063547}. Best is trial 1 with value: 0.604017553269665.


...

Trial 172 finished with value: 0.6037020398755989
Best trial so far: 133, value: 0.6032667579901602


[I 2025-11-25 17:37:56,359] Trial 173 finished with value: 0.6033436163041602 and parameters: {'learning_rate': 0.0400115256490108, 'max_depth': 4, 'n_estimators': 172, 'subsample': 0.6048245335096665, 'colsample_bytree': 0.9280400884961644, 'min_child_weight': 5, 'reg_lambda': 1.7928140900395328, 'reg_alpha': 0.0014035372171593515, 'gamma': 4.257458538608993, 'other_eval_metric': 'error', 'scale_pos_weight': 2.349724931759059}. Best is trial 133 with value: 0.6032667579901602.


Trial 173 finished with value: 0.6033436163041602
Best trial so far: 133, value: 0.6032667579901602


[I 2025-11-25 17:37:58,119] Trial 174 finished with value: 0.6033308228408856 and parameters: {'learning_rate': 0.03241393135649866, 'max_depth': 4, 'n_estimators': 171, 'subsample': 0.6097274097003743, 'colsample_bytree': 0.9270271647826969, 'min_child_weight': 5, 'reg_lambda': 1.8330057231679562, 'reg_alpha': 0.0034295343037598824, 'gamma': 4.325288859377845, 'other_eval_metric': 'error', 'scale_pos_weight': 2.3499067259598454}. Best is trial 133 with value: 0.6032667579901602.


Trial 174 finished with value: 0.6033308228408856
Best trial so far: 133, value: 0.6032667579901602


# Over/under 74.5 points
### Test best params & check how calibrated they are

In [13]:
# Trial 260 finished with value: 0.689417728956386
bestParams = {'learning_rate': 0.03959299078566123, 'max_depth': 2, 'n_estimators': 116, 'subsample': 0.8880039974695518, 'colsample_bytree': 0.7735993499483397, 'min_child_weight': 5, 'reg_lambda': 3.956383913116503, 'reg_alpha': 0.004923916196709604, 'gamma': 0.14719743166780305, 'other_eval_metric':'error'}

# Trial 1039 finished with value: 0.6893976627336035 (This is best 10/28/25)
bestParams = {'learning_rate': 0.024741164833444476, 'max_depth': 6, 'n_estimators': 114, 'subsample': 0.7784223531891586, 'colsample_bytree': 0.6247780028956175, 'min_child_weight': 8, 'reg_lambda': 0.3758603208997029, 'reg_alpha': 1.9149256323607524e-05, 'gamma': 0.6465723884007113, 'other_eval_metric':'error'}

# Best trial so far: 598, value: 0.6885969264344991 (earlier odds may be better)
#bestParams =  {'learning_rate': 0.03792849951262246, 'max_depth': 3, 'n_estimators': 368, 'subsample': 0.7745882502364526, 'colsample_bytree': 0.5783370786187912, 'min_child_weight': 19, 'reg_lambda': 0.010214403456232415, 'reg_alpha': 6.223306902226365e-08, 'gamma': 2.701023147596355, 'other_eval_metric': 'auc'}
# Best trial so far: 247, value: 0.6887602022167105 (earlier odds may be better)
# bestParams = {'learning_rate': 0.019121375057154093, 'max_depth': 3, 'n_estimators': 315, 'subsample': 0.7753063635332988, 'colsample_bytree': 0.5380834314337124, 'min_child_weight': 19, 'reg_lambda': 0.017035757489041285, 'reg_alpha': 3.9505847235869564e-07, 'gamma': 2.731862289717825, 'other_eval_metric': 'auc'}
# Best trial so far: 833, value: 0.24694652319936383
# bestParams = {'learning_rate': 0.0024259699080358826, 'max_depth': 4, 'n_estimators': 634, 'subsample': 0.7193827561877674, 'colsample_bytree': 0.6555067196693295, 'min_child_weight': 7, 'reg_lambda': 1.1362647290828565, 'reg_alpha': 2.2153445039250114e-05, 'gamma': 0.9318701485711431, 'other_eval_metric': 'error'}
#bestParams = {'learning_rate': 0.0019006977434359348, 'max_depth': 4, 'n_estimators': 610, 'subsample': 0.7028468982467145, 'colsample_bytree': 0.6488958998006723, 'min_child_weight': 7, 'reg_lambda': 0.9953548837525594, 'reg_alpha': 1.9108889052195762e-05, 'gamma': 0.927607058631077, 'other_eval_metric': 'error'}

classifier, calibrator = objective(None, bestParams)


# H_won

In [13]:
# Trial 19 finished with value: 0.6768958055818088
bestParams = {'learning_rate': 0.05937265565289764, 'max_depth': 4, 'n_estimators': 962, 'subsample': 0.9765638360100586, 'colsample_bytree': 0.828105890020878, 'min_child_weight': 1, 'reg_lambda': 1.5424681954224457, 'reg_alpha': 0.010763953741291608, 'gamma': 0.7340844790769335, 'other_eval_metric': 'logloss'}

# (No odds) Best trial so far: 12, value: 0.6846107052359698
bestParams =  {'learning_rate': 0.07901247593312617, 'max_depth': 6, 'n_estimators': 980, 'subsample': 0.52238598254258, 'colsample_bytree': 0.6686876677189448, 'min_child_weight': 9, 'reg_lambda': 0.8569450192455178, 'reg_alpha': 1.015742671304129e-06, 'gamma': 0.29732139055797724, 'other_eval_metric': 'error'}

# Best trial so far: 0, value: 0.678127921090242
bestParams = {'learning_rate': 0.002610119419945401, 'max_depth': 7, 'n_estimators': 556, 'subsample': 0.8956572505213283, 'colsample_bytree': 0.7482086886780271, 'min_child_weight': 9, 'reg_lambda': 3.538971364161639, 'reg_alpha': 4.0109507428180954e-07, 'gamma': 1.284790152179896, 'other_eval_metric': 'logloss'}

# Best trial so far: 243, value: 0.6764620882672173
bestParams = {'learning_rate': 0.06552542856853143, 'max_depth': 4, 'n_estimators': 504, 'subsample': 0.9243521642791432, 'colsample_bytree': 0.9681206815300454, 'min_child_weight': 10, 'reg_lambda': 0.040428841065212245, 'reg_alpha': 2.0642270469027124e-07, 'gamma': 3.3662217527056506, 'other_eval_metric': 'error'}

# Best trial so far: 779, value: 0.676456157067713
bestParams = {'learning_rate': 0.05082519468408648, 'max_depth': 4, 'n_estimators': 581, 'subsample': 0.9298260456307847, 'colsample_bytree': 0.9491219872803598, 'min_child_weight': 10, 'reg_lambda': 0.061620009454451764, 'reg_alpha': 6.476257397545602e-07, 'gamma': 3.040396877396999, 'other_eval_metric': 'error'}

# Best trial so far: 229, value: 0.6755795401158257
# NOt as good as last also with 2nd set of params
# bestParams = {'learning_rate': 0.011423595072040068, 'max_depth': 6, 'n_estimators': 538, 'subsample': 0.8486588334630327, 'colsample_bytree': 0.9630317901693166, 'min_child_weight': 18, 'reg_lambda': 0.5985992972193335, 'reg_alpha': 3.588065911732203e-08, 'gamma': 3.2454765730936277, 'other_eval_metric': 'error'}
# Best trial so far: 196, value: 0.6758538817403664
bestParams = {'learning_rate': 0.013432345582953554, 'max_depth': 5, 'n_estimators': 553, 'subsample': 0.8135368296732948, 'colsample_bytree': 0.9886033137478911, 'min_child_weight': 17, 'reg_lambda': 0.6306261897249343, 'reg_alpha': 8.221765294374316e-08, 'gamma': 2.960100601911516, 'other_eval_metric': 'error'}

# -------------------------------------
# ------ After ablation study ---------
# -------------------------------------
# Best trial so far: 610, value: 0.6520162656292539
bestParams = {'learning_rate': 0.024164837490240742, 'max_depth': 4, 'n_estimators': 607, 'subsample': 0.8779362187982347, 'colsample_bytree': 0.7503737143178703, 'min_child_weight': 1, 'reg_lambda': 0.28982275176022393, 'reg_alpha': 0.11135057038778484, 'gamma': 2.283863682790501, 'other_eval_metric': 'error', 'scale_pos_weight': 2.3267091398351285}
# Best trial so far: 1685, value: 0.6520114473027023 (UHH THIS HAS scale Pos weight)
bestParams = {'learning_rate': 0.029440079090217477, 'max_depth': 4, 'n_estimators': 354, 'subsample': 0.8627035323451269, 'colsample_bytree': 0.7030795000344829, 'min_child_weight': 1, 'reg_lambda': 0.1523906904903187, 'reg_alpha': 0.31693626692992394, 'gamma': 0.0886472931128312, 'other_eval_metric': 'error', 'scale_pos_weight': 2.320055989399337}

# Best trial so far: 696, value: 0.6518263980957888
bestParams = {'learning_rate': 0.03306429015169762, 'max_depth': 5, 'n_estimators': 916, 'subsample': 0.8914081947102843, 'colsample_bytree': 0.9739736235201611, 'min_child_weight': 4, 'reg_lambda': 0.23711387567570685, 'reg_alpha': 0.007849904994497994, 'gamma': 1.624390068599687, 'other_eval_metric': 'error'}
# Best trial so far: 382, value: 0.6518846754308942
bestParams = {'learning_rate': 0.049443307557736633, 'max_depth': 5, 'n_estimators': 651, 'subsample': 0.9474096956994965, 'colsample_bytree': 0.9885897683353102, 'min_child_weight': 4, 'reg_lambda': 0.7142429371514274, 'reg_alpha': 5.768480370385418e-05, 'gamma': 2.2930754796423085, 'other_eval_metric': 'error'}

# WITH ACTUAL RIGHT x_col
# Best trial so far: 1752, value: 0.6514460441561187
bestParams = {'learning_rate': 0.01211393992881223, 'max_depth': 8, 'n_estimators': 239, 'subsample': 0.515182030892653, 'colsample_bytree': 0.8736403555997476, 'min_child_weight': 15, 'reg_lambda': 19.884640785535865, 'reg_alpha': 0.04387815129125221, 'gamma': 3.7858687464734806, 'other_eval_metric': 'error'}
# Best trial so far: 1098, value: 0.6514511773305454
bestParams = {'learning_rate': 0.01275152838528136, 'max_depth': 8, 'n_estimators': 239, 'subsample': 0.5156357362077864, 'colsample_bytree': 0.8628506398897919, 'min_child_weight': 15, 'reg_lambda': 14.956580932551363, 'reg_alpha': 0.28789213694688764, 'gamma': 4.996478096611755, 'other_eval_metric': 'error'}


classifier, calibrator = objective(None, bestParams)


# G5

In [42]:

bestParams = {'learning_rate': 0.001668893223401683, 'max_depth': 3, 'n_estimators': 282, 'subsample': 0.6121824533078637, 'colsample_bytree': 0.7024085497833387, 'min_child_weight': 5, 'reg_lambda': 0.5161830294412345, 'reg_alpha': 0.00036250386386460807, 'gamma': 0.5349646449008502, 'other_eval_metric':'error'}
# Best trial so far: 426, value: 0.6258810119230529
bestParamns = {'learning_rate': 0.0036011167018495695, 'max_depth': 3, 'n_estimators': 197, 'subsample': 0.6467469679915824, 'colsample_bytree': 0.9997302821616035, 'min_child_weight': 8, 'reg_lambda': 10.067282476830579, 'reg_alpha': 0.001991609177273599, 'gamma': 0.030167521839034227, 'other_eval_metric':'error'}
# Best trial so far: 166, value: 0.6252501968435211
bestParams = {'learning_rate': 0.027266036120905248, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.7059240984039487, 'colsample_bytree': 0.7240989513964091, 'min_child_weight': 6, 'reg_lambda': 6.510317449049249, 'reg_alpha': 0.006093033379060273, 'gamma': 0.7676383649991234, 'other_eval_metric':'error', 'scale_pos_weight': 2.3015646015241678}
# Best trial so far: 41, value: 0.627195472800645
#bestParams = {'learning_rate': 0.022198637955711826, 'max_depth': 4, 'n_estimators': 840, 'subsample': 0.6978311818151439, 'colsample_bytree': 0.8913667292100208, 'min_child_weight': 17, 'reg_lambda': 9.993216991868618, 'reg_alpha': 0.17562450749094413, 'gamma': 3.0313640982489645, 'other_eval_metric': 'aucpr'}

#Trial 41 finished with value: 0.6274301292328124
#bestParams = {'learning_rate': 0.0014117009434666905, 'max_depth': 6, 'n_estimators': 504, 'subsample': 0.7631716022434903, 'colsample_bytree': 0.8146818918258596, 'min_child_weight': 14, 'reg_lambda': 2.61626325093829, 'reg_alpha': 1.4018938876036097e-06, 'gamma': 1.8826781662699252, 'other_eval_metric': 'aucpr'}

# After adding params
# Best trial so far: 35, value: 0.6277626037617653
#bestParams = {'learning_rate': 0.010481591170105551, 'max_depth': 10, 'n_estimators': 934, 'subsample': 0.5927564139885113, 'colsample_bytree': 0.8578145346788956, 'min_child_weight': 12, 'reg_lambda': 12.603374199059594, 'reg_alpha': 0.004230970306837592, 'gamma': 4.247732383030352, 'other_eval_metric': 'auc'}

# (H_won params) Best trial so far: 292, value: 0.6264107968655782
# Best trial so far: 444, value: 0.6267165932732717
bestParams = {'learning_rate': 0.004479686381666177, 'max_depth': 3, 'n_estimators': 535, 'subsample': 0.7732732172099207, 'colsample_bytree': 0.5904808330110781, 'min_child_weight': 1, 'reg_lambda': 0.012661455439150244, 'reg_alpha': 6.46892444779436e-08, 'gamma': 0.1797034230023018, 'other_eval_metric': 'auc', 'scale_pos_weight': 2.3015646015241678}

# Best trial so far: 354, value: 0.6184724713491545
# bestParams = {'learning_rate': 0.027876419041002794, 'max_depth': 3, 'n_estimators': 820, 'subsample': 0.8043666420425214, 'colsample_bytree': 0.6738382954212226, 'min_child_weight': 2, 'reg_lambda': 1.0615283582491144, 'reg_alpha': 2.9782611760046823e-05, 'gamma': 2.4369076963186282, 'other_eval_metric': 'error', 'scale_pos_weight': 2.3389311223780127}

# -------------------------------------
# ------ After ablation study ---------
# -------------------------------------
# UM THESE DIDNT ORIGINALLY HAVE SCALE POS WEIGHT

# Best trial so far: 1196, value: 0.6031456944290677
bestParams = {'learning_rate': 0.0011431945173079065, 'max_depth': 7, 'n_estimators': 309, 'subsample': 0.6193058468861977, 'colsample_bytree': 0.9421836011826511, 'min_child_weight': 4, 'reg_lambda': 0.762042536552626, 'reg_alpha': 4.588255117126174e-07, 'gamma': 4.832387617338556, 'other_eval_metric': 'error'}

# Best trial so far: 1333, value: 0.6031358387570223
bestParams = {'learning_rate': 0.0018815397869461337, 'max_depth': 7, 'n_estimators': 454, 'subsample': 0.6157846453213903, 'colsample_bytree': 0.9239558342909807, 'min_child_weight': 5, 'reg_lambda': 0.5236425241123049, 'reg_alpha': 3.299273960626099e-05, 'gamma': 4.572846434659016, 'other_eval_metric': 'error'}



classifier, calibrator = objective(None, bestParams)



# G4

In [14]:
# Best trial so far: 21, value: 0.69035325703922
bestParams = {'learning_rate': 0.0036972911822452745, 'max_depth': 6, 'n_estimators': 461, 'subsample': 0.837948395479078, 'colsample_bytree': 0.7464564955294609, 'min_child_weight': 9, 'reg_lambda': 1.5015183342270104, 'reg_alpha': 0.008327405442157302, 'gamma': 0.4165157049551762, 'other_eval_metric': 'logloss'}


# BAD Trial 1324 finished with value: 0.66363371914504
# bestParams = {'learning_rate': 0.004196523119054237, 'max_depth': 7, 'n_estimators': 948, 'subsample': 0.5418589299331488, 'colsample_bytree': 0.6109910817939306, 'min_child_weight': 6, 'reg_lambda': 0.12499138337817949, 'reg_alpha': 0.3933365552838541, 'gamma': 3.069263997186816, 'other_eval_metric': 'error'}
# BAD Best trial so far: 1564, value: 0.6635101927157276
# bestParams = {'learning_rate': 0.0022864798714392924, 'max_depth': 7, 'n_estimators': 447, 'subsample': 0.6561306634507178, 'colsample_bytree': 0.6205386352886686, 'min_child_weight': 1, 'reg_lambda': 0.17191287633035035, 'reg_alpha': 0.37850611958819935, 'gamma': 4.424027686126811, 'other_eval_metric': 'error'}

# Trial 5 finished with value: 0.6668067518550506
bestParams = {'learning_rate': 0.29978032406177285, 'max_depth': 9, 'n_estimators': 102, 'subsample': 0.6241784202447722, 'colsample_bytree': 0.5786167862043247, 'min_child_weight': 10, 'reg_lambda': 0.17761598537918127, 'reg_alpha': 0.012857595281312247, 'gamma': 0.027314786005158687, 'other_eval_metric': 'auc'}

# Trial 184 finished with value: 0.6667294256662116
bestParams = {'learning_rate': 0.0027548463523250607, 'max_depth': 9, 'n_estimators': 538, 'subsample': 0.7195716951811234, 'colsample_bytree': 0.5294600322240618, 'min_child_weight': 13, 'reg_lambda': 4.586385852367849, 'reg_alpha': 0.02340998364616693, 'gamma': 4.609922376451254, 'other_eval_metric': 'aucpr'}
bestParams = {'learning_rate': 0.0922281098253217, 'max_depth': 9, 'n_estimators': 183, 'subsample': 0.8486376951337543, 'colsample_bytree': 0.5860672125031787, 'min_child_weight': 17, 'reg_lambda': 1.382584982058371, 'reg_alpha': 4.689260503629778e-07, 'gamma': 2.403968278530627, 'other_eval_metric': 'auc'}
# After SMOTE
# Trial 18 finished with value: 0.6668979600593431

bestParams = {'learning_rate': 0.13304743286015677, 'max_depth': 5, 'n_estimators': 844, 'subsample': 0.9237166179399163, 'colsample_bytree': 0.5650385017853643, 'min_child_weight': 16, 'reg_lambda': 0.1995848855545275, 'reg_alpha': 0.8803236086367877, 'gamma': 4.451798087570364, 'other_eval_metric': 'auc'}
classifier, calibrator = objective(None, bestParams)



# G45

In [33]:
# Best trial so far: 26, value: 0.6031041690958991


# Best trial so far: 189, value: 0.6021270598767348
bestParams = {'learning_rate': 0.06277576241977738, 'max_depth': 3, 'n_estimators': 723, 'subsample': 0.6920574475414543, 'colsample_bytree': 0.7152852499939504, 'min_child_weight': 12, 'reg_lambda': 13.476100594665269, 'reg_alpha': 0.00012886263429678392, 'gamma': 0.6912806584652373, 'other_eval_metric': 'auc'}

# Best trial so far: 2523, value: 0.6021068917261723
bestParams = {'learning_rate': 0.019571701678877305, 'max_depth': 3, 'n_estimators': 524, 'subsample': 0.5163595833889137, 'colsample_bytree': 0.938425232158752, 'min_child_weight': 19, 'reg_lambda': 0.19639593254896393, 'reg_alpha': 3.631358417749485e-06, 'gamma': 2.509164327581109, 'other_eval_metric': 'auc'}

# No SMOTE
bestParams = {'learning_rate': 0.0022373251863992403, 'max_depth': 3, 'n_estimators': 748, 'subsample': 0.9510905446342681, 'colsample_bytree': 0.8181027686655234, 'min_child_weight': 4, 'reg_lambda': 0.015884275778326674, 'reg_alpha': 0.011230742786315313, 'gamma': 4.90491075278029, 'other_eval_metric': 'error'}

classifier, calibrator = objective(None, bestParams)


### Save classifier & calibrator

In [212]:
# appendStr = "accuracy_nov8"
appendStr = "g5_nov25"
pickle.dump(classifier, open(f"models/ttennis_classifier_{appendStr}.pkl", 'wb'))
pickle.dump(calibrator, open(f"models/ttennis_calibrator_{appendStr}.pkl", "wb"))


In [41]:
with open('ttennisData/ttennis_classifierProdOct25.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open('ttennisData/ttennis_calibratorProdOct25.pkl', 'rb') as f:
    calibrator = pickle.load(f)

### Calculate ROC, AUC

In [47]:
from sklearn.metrics import accuracy_score, roc_auc_score

perf_conts_res = classifier.predict_proba(perf_conts)[:,1]
perf_conts_calibrated = calibrator.predict(perf_conts_res)
print(y_prediction)
# Ignore lower bound if predicting 5 games (G5)
include_lower_bound = 0 # Include lower bound
if y_prediction in ['G5', 'G4', 'G45', 'G54']:
    print(y_prediction)
    include_lower_bound = 1.0 # Dont include lower bound
    perf_conts_calibrated = perf_conts_calibrated + 0.2

# IGNORE CALIBRATION
# perf_conts_res = classifier.predict(perf_conts)
perf_conts_calibrated_2d = np.stack([
    1 - perf_conts_calibrated,  # Index 0: P(class=0)
    perf_conts_calibrated       # Index 1: P(class=1)
], axis=1)
perf_conts_calibrated = perf_conts_calibrated_2d

# Filter out where near
print(perf_conts_calibrated.shape)
# print(perf_conts_calibrated)
# Limit usually 0.65-0.7
limit = 0.0

lower_bound = 0.5 - limit - include_lower_bound
upper_bound = 0.5 + limit
# Extract positive class probabilities
positive_probs = perf_conts_calibrated[:, 1]
# positive_probs = perf_conts_calibrated

# Filter out records within ±0.03 of 0.5
mask = ((positive_probs < lower_bound) | (positive_probs > upper_bound))
# Additional mask filters if needed (Ex: G5 prediction in addition to the original accuracy measure)
# mask = (mask == 1) & (test_performance_df['G5'] == 1)
perf_conts_calibrated = perf_conts_calibrated[mask]
perf_y_col_calibrated = perf_y_col[mask]
perf_y_col_calibrated = perf_y_col_calibrated.ravel()
print(perf_y_col_calibrated.shape)


perf_y_eval = np.argmax(perf_conts_calibrated, axis=1)
# print(perf_conts_calibrated)
# print(perf_y_col_calibrated)
print(perf_y_eval.shape)

print('ROC AUC (higher is better): ',  roc_auc_score(perf_y_col_calibrated, perf_conts_calibrated[:,1], multi_class='ovr'), 'Accuracy', accuracy_score(perf_y_col_calibrated, perf_y_eval))
print(f"Raw probs mean: {np.mean(perf_conts_res):.3f}, std: {np.std(perf_conts_res):.3f}")
print(f"Calibrated probs mean: {np.mean(perf_conts_calibrated):.3f}, std: {np.std(perf_conts_calibrated):.3f}")
print(f"Fraction >0.5 (calibrated): {np.mean(perf_conts_calibrated > 0.5):.2%}")
# pcc_tensor = torch.FloatTensor(perf_conts_calibrated[:,0])

# print(perf_conts_calibrated)


G5
G5
(10000, 2)
(5838,)
(5838,)
ROC AUC (higher is better):  0.5148927677688722 Accuracy 0.3069544364508393
Raw probs mean: 0.500, std: 0.000
Calibrated probs mean: 0.500, std: 0.012
Fraction >0.5 (calibrated): 50.00%


#### from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt

# Assuming y_true is true labels and y_prob is predicted probabilities for one class
prob_true, prob_pred = calibration_curve(perf_y_col_calibrated, perf_y_eval, n_bins=10)
plt.plot(prob_pred, prob_true, marker='o', label='Calibration Curve')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Calibration Curve')
plt.legend()
plt.show()
print("Brier Score (lower is better):", brier_score_loss(perf_y_col_calibrated, perf_y_eval))

#### Sanity check

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE().fit_resample(conts_train, y_col)
print(X_resampled.shape)
print(conts_train.shape)
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay


X_train, X_test, y_train, y_test = train_test_split(conts_train, y_col, test_size=0.1, shuffle=False)
sample_weight = compute_sample_weight('balanced', y_train)
model = XGBClassifier(random_state=42, scale_pos_weight=1.6)
model.fit(X_train, y_train, sample_weight=sample_weight)
y_pred = model.predict(X_test)
# Evaluate the model
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
print('Accuracy', accuracy_score(y_test, y_pred))
disp.plot()
plt.show()
# fig, ax = plt.subplots(figsize=(8, 8))
# plot_confusion_matrix(model, X_test, y_test, cmap=plt.cm.Blues, ax=ax);


In [ ]:
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(conts_train, y_col, test_size=0.1, shuffle=False)

xgboster_focal = imb_xgb(special_objective='focal')
CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
CV_focal_booster.fit(X_train, y_train)





## Validate

In [124]:
def makePrediction(df, x_col, model_name, exclude_lower_bound, combine_mask=None, return_mask=False, limit=0.05, perf_conts_shift=0, metric="74.5"):
    """
    df: copy of datafame

    X_col: List of columns to filter from df

    model_name: name of model in /models/ directory. Ex: 'ttennis' for ttennis_classifier, ttennis_calibrator

    exclude_lower_bound: Set to 1.0 to exclude lower bound. Used for predicting G5

    combine_mask: If defined, this will combine the mask passed in with the one generated

    return_mask: True if you want to return mask and not send any notifications. False if you want to apply and send notifications

    limit: Primary means of defining a lower/upper bounds. limit of 0.05 will exclude ranges inside 0.05 of 0.5.

    perf_conts_shift: used alongside exclude_lower_bound=1.0 for models that are making unconfident predictions, this pushes one side to the other essentially

    metric: What's in the notification. example, over/under <metric>. Example: 74.5

    Generally for adding a new model you'll need a few things:
    returns mask which can be joined with mask = (mask1 == 1) & (mask2 == 1)
        https://stackoverflow.com/questions/15579260/how-to-combine-multiple-numpy-masks
    """
    identifier_col = ['Date', 'Player1_Name', 'Player2_Name', 'Player1', 'Player2']
    # Get upcoming data
    # dfUpcoming = df[df['Total_Score'] == 0]
    dfUpcoming = df # This is just applied to all data since theres no 

    xTrue = (dfUpcoming['H_won'])
    xTrueTwo = dfUpcoming['G5'] == 1 # True when G5 was played. Combined with xTrue to get accurate game result
    xId = dfUpcoming[identifier_col]
    x = dfUpcoming[x_col]

    conts = np.stack([x[col].values for col in list(x.columns)], 1)
    idConts = np.stack([xId[col].values for col in list(xId.columns)], 1)
    # xTrue = np.stack([xTrue[col].values for col in list(xTrue.columns)], 1)

    # Import saved classifier, calibrator (will error out if DNE)
    with open(f'models/ttennis_classifier_{model_name}.pkl', 'rb') as f:
        classifier = pickle.load(f)

    with open(f'models/ttennis_calibrator_{model_name}.pkl', 'rb') as f:
        calibrator = pickle.load(f)

    # Apply the model to the upcoming data
    perf_conts_res = classifier.predict_proba(conts)[:,1]
    perf_conts_calibrated = calibrator.predict(perf_conts_res)
    perf_conts_calibrated = perf_conts_calibrated + perf_conts_shift
    perf_conts_calibrated_2d = np.stack([
        1 - perf_conts_calibrated,  # Index 0: P(class=0)
        perf_conts_calibrated       # Index 1: P(class=1)
    ], axis=1)
    perf_conts_calibrated = perf_conts_calibrated_2d
    positive_probs = perf_conts_calibrated[:, 1]

    # Define bounds around 0.5
    lower_bound = 0.5 - limit - exclude_lower_bound
    upper_bound = 0.5 + limit

    # Filter lower/upper bound
    mask = (positive_probs < lower_bound) | (positive_probs > upper_bound)
    # Combine masks if combine_mask is not None
    if combine_mask is not None:
        combined_mask = (mask == 1) & (combine_mask == 1)
        mask = combined_mask

    perf_conts_calibrated = perf_conts_calibrated[mask]
    conts_id = idConts[mask]
    xTrue = xTrue[mask]
    xTrueTwo = xTrueTwo[mask]
    print(conts_id.shape)
    print(xTrue.shape)
    print(xTrueTwo.shape)
    # return mask if return_mask == True
    if return_mask == True:
        return mask

    # ---------------------------------------
    # ---------- Report results -------------
    # ---------------------------------------


    currentDatetimeEDT = (datetime.now(timezone.utc)).astimezone(ZoneInfo("America/New_York"))
    #datetime.utcfromtimestamp(datetime.utcnow().timestamp())
    currentDatetimeEDTPlusFourHours = currentDatetimeEDT + timedelta(hours=7)
    # df.to_csv(f'./TTCupSlidingTESTUPC.csv')

    print(f"CurrentDateTimeEDT {currentDatetimeEDT.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"CurrentDateTimeEDT+4 {currentDatetimeEDTPlusFourHours.strftime('%Y-%m-%d %H:%M:%S')}")

    hit = 0
    
    for i in range(0,len(perf_conts_calibrated)):
        home = perf_conts_calibrated[i][1] > perf_conts_calibrated[i][0]

        # Assuming conts_id[i][0] is a Unix timestamp in seconds
        utc_dt = datetime.fromtimestamp(int(conts_id[i][0]), tz=timezone.utc)
        edt_dt = utc_dt.astimezone(ZoneInfo("America/New_York"))
        dts = edt_dt.strftime('%Y-%m-%d %H:%M:%S')

        homeAway = "home" if home == True else "away"
        homeActual = "---"
        # if 5 games played..
        if xTrueTwo.iloc[i] == True:
            homeActual = "home" if xTrue.iloc[i] == True else "away"
        
        # resString = f"Date: {dts}, {rightLeft} {metric}, {conts_id[i][1]} vs {conts_id[i][2]}, SearchString: {searchString}\n"
        resString = f"{dts} {conts_id[i][1]} vs {conts_id[i][2]}"
        messageString = edt_dt.strftime("%A %-I:%M %p") + f", {homeAway} {metric}"

        edt_dt_plus = edt_dt + timedelta(minutes=4)
        print(f"Actual: {homeActual}- {resString} - {messageString}")
        if homeAway == homeActual:
            hit = hit + 1
        # Skip if upcoming event already happened or if > 4 hours from current time
#        if currentDatetimeEDT >= edt_dt_plus or currentDatetimeEDTPlusFourHours <= edt_dt:
#            print(f"SKIPPING {currentDatetimeEDT > edt_dt} {currentDatetimeEDTPlusFourHours < edt_dt}")
#            print(f"FIRST {currentDatetimeEDT}    {edt_dt}      {currentDatetimeEDTPlusFourHours}")
#            print(resString)
#            continue

        # Skip if already sent notification, otherwise add to TTNotif.csv
        row = [ dts, resString ]
        chkRow = np.array(row)
    print(f"{hit} out of {len(perf_conts_calibrated)}. {hit / len(perf_conts_calibrated)}")

    

In [125]:
test_performance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 81402 to 81701
Data columns (total 54 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        300 non-null    int64  
 1   Date              300 non-null    int64  
 2   Player1           300 non-null    int64  
 3   Player2           300 non-null    int64  
 4   Sets_P1           300 non-null    float64
 5   Sets_P2           300 non-null    float64
 6   G1_P1             300 non-null    float64
 7   G1_P2             300 non-null    float64
 8   G2_P1             300 non-null    float64
 9   G2_P2             300 non-null    float64
 10  G3_P1             300 non-null    float64
 11  G3_P2             300 non-null    float64
 12  G4_P1             300 non-null    float64
 13  G4_P2             300 non-null    float64
 14  G5_P1             300 non-null    float64
 15  G5_P2             300 non-null    float64
 16  Win_P1            300 non-null    floa

In [132]:

print(type(test_performance_df))
# (512/2k) with limit 0, perf_conts_shift = 0.17 at 0.384 accuracy
g5_x_col = ['G5_P1', 'Total_Allowed_P2', 'G5_P2', 'elo_P2', 'Total_Allowed_P1', 'G4_P1', 'Total_Avg_P2', 'D_Odds', 'Total_Avg_P1', 'elo_P1', 'D_G2', 'G1_P1', 'G3_P1', 'pythagorean_P1', 'pythagorean_P2']
g5_mask = makePrediction(test_performance_df, g5_x_col, "g5_nov8", 1.0, None, True, 0.0, 0.17, "G5")
accuracy_x_col = ['D_Odds', 'pythagorean_P1', 'pythagorean_P2', 'elo_P2', 'elo_P1', 'D_G3', 'D_G1', 'Sets_P2', 'Win_P1', 'Total_Allowed_P1', 'D_G2', 'Win_P2', 'Total_Allowed_P2', 'Sets_P1']
# accuracy_mask is on dfUpcoming
# (195/2k) with G5, ROC AUC (higher is better):  0.6892887205387205 Accuracy 0.6820512820512821
makePrediction(test_performance_df, accuracy_x_col, "accuracy_nov8", 0.0, g5_mask, False, 0.12, 0.0, "Accuracy")




<class 'pandas.core.frame.DataFrame'>
(533, 5)
(533,)
(533,)
(150, 5)
(150,)
(150,)
CurrentDateTimeEDT 2025-11-11 20:25:27
CurrentDateTimeEDT+4 2025-11-12 03:25:27
Actual: away- 2025-10-30 16:50:00 Ales Brandtl Jr vs Martin Topol - Thursday 4:50 PM, away Accuracy
Actual: ---- 2025-10-30 17:30:00 Kuba Golaszewski vs Karol Sulkowski - Thursday 5:30 PM, away Accuracy
Actual: away- 2025-10-30 19:30:00 Kacper Turecki vs Karol Sulkowski - Thursday 7:30 PM, home Accuracy
Actual: away- 2025-10-31 04:35:00 Franciszek Jastrzebowski vs Tomasz Witkowski - Friday 4:35 AM, away Accuracy
Actual: ---- 2025-10-31 05:05:00 Artur Moroz vs Karol Guzy - Friday 5:05 AM, away Accuracy
Actual: ---- 2025-10-31 05:20:00 Jan Vonasek vs Jan Simecek - Friday 5:20 AM, home Accuracy
Actual: away- 2025-10-31 06:05:00 Artur Moroz vs Tomasz Witkowski - Friday 6:05 AM, away Accuracy
Actual: away- 2025-10-31 06:30:00 Pedro Sanchez vs Hajji Hamza - Friday 6:30 AM, away Accuracy
Actual: ---- 2025-10-31 06:35:00 Franciszek 